## OJS Submission Export

Code for generating a CSV file for all submissions during a given date range from a local version of the database. Runs on Python3. Additional authors are printed within "{}".

In [ ]:
import pymysql.cursors

In [ ]:
# Connect to the database
db = pymysql.connect(host='localhost',
                            user='',
                            password='',
                            db='',
                            cursorclass=pymysql.cursors.DictCursor)

In [ ]:
journal_id = 10

In [ ]:
# For submission: ID | Title | Author | Abstract (if possible) |  Section (if possible)

In [ ]:
import time   
start_date = time.strftime('2016-02-15 00:00:00')
end_date = time.strftime('2016-03-22 13:00:00')

In [ ]:
# Article ID, Article Author, Article Title, Section ID

with db.cursor() as cursor:
    sql = ("""SELECT articles.article_id, article_settings.setting_value, articles.section_id 
        FROM articles 
        LEFT JOIN (article_settings) 
            on (articles.article_id = article_settings.article_id)
        WHERE articles.journal_id = %s 
            AND articles.date_submitted BETWEEN %s and %s 
            AND article_settings.setting_name LIKE 'cleanTitle'""")
    cursor.execute(sql, (journal_id, start_date, end_date))
    result = cursor.fetchall()

In [ ]:
# result

In [ ]:
for article in result:
    article_id = article.get('article_id')
    section_id = article.get('section_id')
    with db.cursor() as cursor:
        sql1 = """SELECT article_settings.setting_value
        FROM article_settings
        WHERE article_settings.article_id = %s
        AND setting_name LIKE 'abstract'"""
        
        sql2 = """SELECT section_settings.setting_value
        FROM section_settings
        WHERE section_settings.section_id = %s
        AND setting_name LIKE 'title'"""
        
        sql3 = """ SELECT authors.first_name, authors.last_name
        FROM authors
        WHERE authors.submission_id = %s"""
        
        cursor.execute(sql1, (article_id))
        abstract_dict = cursor.fetchone()
#         print(abstract_dict)
        abstract = abstract_dict.get('setting_value')
#         print(abstract)
        cursor.execute(sql2, (section_id))
        section_dict = cursor.fetchone()
        section = section_dict.get('setting_value')
        
        cursor.execute(sql3, (article_id))
        authors_dict = cursor.fetchall()
#         print(authors_dict)
        
        author1 = authors_dict[0]
        firstname = author1['first_name']
        lastname = author1['last_name']
        try:
            additional_authors = authors_dict[1:]
        except:
            pass

        article.update({'abstract': abstract, 'section': section, 'firstname': firstname, 'lastname': lastname, 'additional_authors': additional_authors })
cursor.close()  
    

In [ ]:
# result

In [ ]:
import csv

In [ ]:
keys = result[0].keys()
with open('ITLCP-Submission-2016.csv', 'w') as f:  # Just use 'w' mode in 3.x
    dict_writer = csv.DictWriter(f, keys)
    dict_writer.writeheader()
    dict_writer.writerows(result)